In [1]:
DIR = "norms"

import os
import bilby

import numpy as np

from glob import glob
from scipy import stats
from likelihood import Likelihood

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# emcee variables
nsteps = 10000 # How many steps should each walker take?
nthin  = 10    # How often should the walker save a step?
nprocs = 6     # How many Python processes do you want to allocate?

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Define the parameters prior distributions
priors = dict(
    param_0=bilby.core.prior.Gaussian(1.63, 0.12, "param_0"),

    param_1=bilby.core.prior.Uniform(2.30, 2.40, "param_1"), 
    param_2=bilby.core.prior.Uniform(0, 1e6, "param_2"), 
    param_3=bilby.core.prior.Uniform(-10, 0, "param_3"),
    param_4=bilby.core.prior.Uniform(-1e6, 1e6, "param_4"),

    param_5=bilby.core.prior.Uniform(3.45, 3.55, "param_5"), 
    param_6=bilby.core.prior.Uniform(0, 1e6, "param_6"), 
    param_7=bilby.core.prior.Uniform(-10, 0, "param_7"), 
    param_8=bilby.core.prior.Uniform(-1, 1, "param_8"),
    param_9=bilby.core.prior.Uniform(-1e6, 0, "param_9"), 
    param_10=bilby.core.prior.Uniform(-1e6, 1e6, "param_10"),

    param_11=bilby.core.prior.Uniform(3.50, 3.60, "param_11"), 
    param_12=bilby.core.prior.Uniform(0, 1e6, "param_12"),
    
    param_13=bilby.core.prior.LogNormal(0, 0.05, "param_13"),
    param_14=bilby.core.prior.LogNormal(0, 0.05, "param_14"),
    param_15=bilby.core.prior.LogNormal(0, 0.05, "param_15"),
    param_16=bilby.core.prior.LogNormal(0, 0.069, "param_16"),
    param_17=bilby.core.prior.LogNormal(0, 0.079, "param_17"),
    param_18=bilby.core.prior.LogNormal(0, 0.10, "param_18"),
    param_19=bilby.core.prior.LogNormal(0, 0.10, "param_19"),
    param_20=bilby.core.prior.LogNormal(0, 0.10, "param_20"),
    param_21=bilby.core.prior.LogNormal(0, 0.10, "param_21"),
    param_22=bilby.core.prior.LogNormal(0, 0.10, "param_22"),
    param_23=bilby.core.prior.Uniform(0, 10, "param_23"),
    param_24=bilby.core.prior.Uniform(0, 10, "param_24"),
    param_25=bilby.core.prior.Uniform(0, 10, "param_25"),
    param_26=bilby.core.prior.Uniform(0, 10, "param_26"),
    param_27=bilby.core.prior.Uniform(0, 10, "param_27")
)


In [2]:
# Preparing initial parameters for the walkers
data = []
files = glob( "results/{}/samples_*.txt".format(DIR) )
for file in files:
    data.append( np.loadtxt( file ) )
data = np.concatenate( data, axis=0 )

mean = np.mean( data, axis=0 )

nt = 20
nw = 2 * len(priors)
p0 = np.zeros( (nt, nw, len(priors)) )
for t in range(nt):
    for i in range(nw):
        for j in range(len(priors)):
            if( j not in [1, 5, 11] ): p0[t,i,j] = np.sign(mean[j]) * stats.uniform( abs(mean[j]) * 0.6, abs(mean[j]) * 0.8 ).rvs()
            else: p0[t,i,j] = np.sign(mean[j]) * stats.uniform( abs(mean[j]) * 0.99, abs(mean[j]) * 0.02 ).rvs()

In [3]:
likelihood = Likelihood( "12c_pg.azr", nprocs=nprocs)

result = bilby.run_sampler( likelihood=likelihood, priors=priors, npool=nprocs,
                            sampler="ptemcee", outdir="results/bilby/", label="samples", 
                            pos0=p0, nwalkers=nw, ntemps=nt, clean=True )

22:01 bilby INFO    : Running for label 'samples', output will be saved to 'results/bilby/'
22:01 bilby INFO    : Analysis priors:
22:01 bilby INFO    : param_0=Gaussian(mu=1.63, sigma=0.12, name='param_0', latex_label='param_0', unit=None, boundary=None)
22:01 bilby INFO    : param_1=Uniform(minimum=2.3, maximum=2.4, name='param_1', latex_label='param_1', unit=None, boundary=None)
22:01 bilby INFO    : param_2=Uniform(minimum=0, maximum=1000000.0, name='param_2', latex_label='param_2', unit=None, boundary=None)
22:01 bilby INFO    : param_3=Uniform(minimum=-10, maximum=0, name='param_3', latex_label='param_3', unit=None, boundary=None)
22:01 bilby INFO    : param_4=Uniform(minimum=-1000000.0, maximum=1000000.0, name='param_4', latex_label='param_4', unit=None, boundary=None)
22:01 bilby INFO    : param_5=Uniform(minimum=3.45, maximum=3.55, name='param_5', latex_label='param_5', unit=None, boundary=None)
22:01 bilby INFO    : param_6=Uniform(minimum=0, maximum=1000000.0, name='param_6'

1|0:00:31|nc:2.8e+03|a0:0.00-1.00|swp:0.41-0.77|n:nan<5000|t!nan(+nan,+nan)|q:inf|11.19ms/ev
